In [1]:
import os, sys, psycopg2, pickle
import numpy as np
import warnings; warnings.filterwarnings("ignore")
from natsort import natsorted
from scipy import misc
from PIL import Image, ImageFilter

### Create database table for Regions Clicking feature

In [2]:
regions_dir = '../app/assets/images/2-Regions'
regions_subdirs = [d for d in os.listdir(regions_dir) if not d.startswith('.')]
regions_subdirs = [os.path.join(regions_dir, s, 'cyan') for s in regions_subdirs]
regions_subdirs = natsorted(regions_subdirs)[2:]
print(len(regions_subdirs), regions_subdirs[0])

292 ../app/assets/images/2-Regions/Forebrain - Diencephalon - Anterior group of the posterior tubercular vmat2 neurons/cyan


#### Create blank dict (138, 621, 1406, [])

In [1]:
slices_dict = dict()
slice_shape = (621, 1406)
no_slices = 138
for i in range(0, no_slices):
    slices_dict[i] = np.ndarray(slice_shape, dtype=object)

NameError: name 'np' is not defined

In [65]:
for i in range(0, no_slices):
    for index, x in np.ndenumerate(slices_dict[i]):
        slices_dict[i][index] = list()

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/message.c:4294)
KeyboardInterrupt


#### Populate dict with vals for region matches

In [3]:
def populate_slices_dict(slices_dict, regions_subdirs):
    for i, subdir in enumerate(regions_subdirs):
        try:
            images = natsorted([os.path.join(subdir, image) \
                        for image in os.listdir(subdir)
                        if not image.startswith('.')])
        except:
            continue
        for sli_n, image in enumerate(images):
            im = misc.imread(image, 'L')
            for index, x in np.ndenumerate(im):
                if x:
                    region_name = subdir.split('/')[5]
                    if region_name not in slices_dict[sli_n][index]:
                        slices_dict[sli_n][index].append(region_name)

In [ ]:
try: 
    slices_dict = pickle.load(open('slices_dict.p', 'rb'))
except:
    print('pickle does not exist')

In [ ]:
for i in range(189, len(regions_subdirs)):
    print(i, regions_subdirs[i])
    populate_slices_dict(slices_dict, [regions_subdirs[i]])
    if i % 5 == 0:
        try:
            slices_dict_pickle = pickle.load(open('slices_dict.p', 'rb'))
            slices_dict_pickle.update(slices_dict)
            pickle.dump(slices_dict_pickle, open('slices_dict.p', 'wb'))
        except:
            pickle.dump(slices_dict, open('slices_dict.p', 'wb'))


189 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 1 - Cerebellum - Valvula Cerebellum/cyan
190 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 1 - Interpeduncular Nucleus/cyan
191 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 1 - Locus Coreuleus/cyan
192 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 1 - Oculomotor Nucleus nIV/cyan
193 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 1 - Raphe - Superior/cyan
194 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 1 - RoL-R1/cyan
195 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 1 - Ventrolateral Stripe of Serotonergic Neurons/cyan
196 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 1/cyan
197 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 2 - Anterior Cluster of nV Trigeminal Motorneurons/cyan
198 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 2 - Locus Coreuleus/cyan
199 ../app/assets/images/2-Regions/Hindbrain - Rhombomere 2 - Raphe - Superior/cyan
200 ../app/assets/ima

In [18]:
for key in slices_dict.keys():
    for index, x in np.ndenumerate(slices_dict[key]):
        if x and x != [None]:
            slices_dict[key][index] = ', '.join(x)
        else:
            slices_dict[key][index] = [None]

In [21]:
print(slices_dict[1][0][30])

[None]


#### Init and fill PostgreSQL database with slices_dict vals

In [22]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [23]:
create_table_command = (
"""
    DROP TABLE IF EXISTS regionsClicking;
    CREATE TABLE regionsClicking (
        slice_id SERIAL PRIMARY KEY,
        slice_no INT NOT NULL,
        slice_vals text[][]
    );
""")

In [24]:
try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [25]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [26]:
def create_insert_commands(slices_dict_pickle):
    keys = natsorted(slices_dict_pickle.keys())
    for i, key in enumerate(keys):
        sli = slices_dict_pickle[key].tolist()
        insert_SQL = (
            "INSERT INTO regionsClicking "
            "(slice_no, slice_vals) "
            "VALUES (%s, %s);")
        values = [i, sli]
        yield (insert_SQL, values)

In [27]:
insert_commands = list(create_insert_commands(slices_dict_pickle))

In [29]:
print(np.array(insert_commands[58][1][1]))

[[list([None]) list([None]) list([None]) ..., list([None]) list([None])
  list([None])]
 [list([None]) list([None]) list([None]) ..., list([None]) list([None])
  list([None])]
 [list([None]) list([None]) list([None]) ..., list([None]) list([None])
  list([None])]
 ..., 
 [list([None]) list([None]) list([None]) ..., list([None]) list([None])
  list([None])]
 [list([None]) list([None]) list([None]) ..., list([None]) list([None])
  list([None])]
 [list([None]) list([None]) list([None]) ..., list([None]) list([None])
  list([None])]]


In [30]:
def execute_inserts(insert_commands):
    conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
    cursor = conn.cursor()

    for insert_command in insert_commands:
        try:
            cursor.execute(insert_command[0], insert_command[1])
        except Exception as e:
            print(e)
        
    cursor.close()
    conn.commit()

In [ ]:
execute_inserts(insert_commands)

In [52]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM regionsClicking WHERE slice_id=40')

rows = cursor.fetchall()
for index, item in np.ndenumerate(rows[0][2]):
    if item != '{NULL}':
        print(item)
        break

cursor.close()
conn.close()

Ganglia - Vagal Ganglia
